In [71]:
import pyodbc
import pandas as pd
import warnings
from datetime import datetime

# Suppress all warnings
warnings.filterwarnings("ignore")

### 1. Connecting to the SQL Server: 

In [4]:
Server= 'Sufiyan-Workstation2\SQLEXPRESS03'
Database= 'AdventureWorksDW2022'

In [62]:
# Establishing SQL Server Connection
cons = pyodbc.connect(
        driver='{SQL Server}',
        server=Server,
        port='1433',
        database=Database,
        trusted_connection='yes')

### 2. Loading Relevant Data: 

In [8]:
dim_cust_query = """

SELECT 
  c.customerkey AS CustomerKey, 
  c.firstname + ' ' + lastname AS [Full Name],
  CASE c.gender WHEN 'M' THEN 'Male' WHEN 'F' THEN 'Female' END AS Gender, 
  g.city AS [Customer City] -- Joined in Customer City from Geography Table
FROM 
  dbo.dimcustomer AS c 
  LEFT JOIN dbo.DimGeography AS g ON g.geographykey = c.geographykey 
ORDER BY 
  CustomerKey ASC
"""

dim_cust_df = pd.read_sql(dim_cust_query, cons)

dim_cust_df.head()

,CustomerKey,Full Name,Gender,Customer City
0,11000,Jon Yang,Male,Rockhampton
1,11001,Eugene Huang,Male,Seaford
2,11002,Ruben Torres,Male,Hobart
3,11003,Christy Zhu,Female,North Ryde
4,11004,Elizabeth Johnson,Female,Wollongong


In [9]:
dim_prod_query = """
SELECT 
  p.[ProductKey],  
  p.[EnglishProductName] AS [Product Name], 
  ps.EnglishProductSubcategoryName AS [Sub Category],
  pc.EnglishProductCategoryName AS [Product Category], 
  ISNULL(p.Status, 'Outdated') AS [Product Status] 
FROM 
  [dbo].[DimProduct] AS p 
  LEFT JOIN dbo.DimProductSubcategory AS ps ON ps.ProductSubcategoryKey = p.ProductSubcategoryKey 
  LEFT JOIN dbo.DimProductCategory AS pc ON ps.ProductCategoryKey = pc.ProductCategoryKey 
ORDER BY 
  p.ProductKey ASC
"""

dim_prod_df = pd.read_sql(dim_prod_query, cons)

dim_prod_df.head()

,ProductKey,Product Name,Sub Category,Product Category,Product Status
0,1,Adjustable Race,None,None,Current
1,2,Bearing Ball,None,None,Current
2,3,BB Ball Bearing,None,None,Current
3,4,Headset Ball Bearings,None,None,Current
4,5,Blade,None,None,Current


The Sales data was selected for 2022-2023 and downloaded from the Server along with the budget data

In [100]:
fact_sales_df = pd.read_csv("FACT_InternetSales.csv")

In [101]:
fact_sales_df.head()

,ProductKey,OrderDateKey,CustomerKey,SalesOrderNumber,SalesAmount
0,381,20220101,16942,SO46700,1000.4375
1,375,20220101,15114,SO46701,2181.5625
2,369,20220101,15116,SO46702,2443.3500
3,337,20220101,20576,SO46703,782.9900
4,370,20220101,13059,SO46704,2443.3500


In [103]:
fact_budget_df = pd.read_csv("Budget_Data.csv")
fact_budget_df.head()

,Date,Budget
0,January-22,379665
1,February-22,360220
2,March-22,391676
3,April-22,374843
4,May-22,371747


## 3. Data Cleaning: 

The data is now looked at and then cleaned (missing values check, NaN errors, formats)

### 3.1 Missing values analysis:

#### Customer Database:

In [55]:
dim_cust_df.head()

,CustomerKey,Full Name,Gender,Customer City
0,11000,Jon Yang,Male,Rockhampton
1,11001,Eugene Huang,Male,Seaford
2,11002,Ruben Torres,Male,Hobart
3,11003,Christy Zhu,Female,North Ryde
4,11004,Elizabeth Johnson,Female,Wollongong


In [12]:
print(dim_cust_df.isnull().sum()[dim_cust_df.isnull().sum() > 0])

Series([], dtype: int64)


#### Product Database:

In [56]:
dim_prod_df.head()

,ProductKey,Product Name,Sub Category,Product Category,Product Status
0,1,Adjustable Race,Uncategorized,Uncategorized,Current
1,2,Bearing Ball,Uncategorized,Uncategorized,Current
2,3,BB Ball Bearing,Uncategorized,Uncategorized,Current
3,4,Headset Ball Bearings,Uncategorized,Uncategorized,Current
4,5,Blade,Uncategorized,Uncategorized,Current


In [35]:
unique_product_keys = dim_prod_df['Product Status'].nunique()
print(unique_product_keys)

2


In [36]:
print(dim_prod_df['Product Status'].unique())

['Current' 'Outdate']


In [37]:
dim_prod_df = dim_prod_df[dim_prod_df['Product Status'] != 'Outdate']

In [38]:
dim_prod_df.head()

,ProductKey,Product Name,Sub Category,Product Category,Product Status
0,1,Adjustable Race,None,None,Current
1,2,Bearing Ball,None,None,Current
2,3,BB Ball Bearing,None,None,Current
3,4,Headset Ball Bearings,None,None,Current
4,5,Blade,None,None,Current


In [39]:
print(dim_prod_df.isnull().sum()[dim_prod_df.isnull().sum() > 0])

Sub Category        209
Product Category    209
dtype: int64


In [40]:
dim_prod_df['Product Category'].fillna('Uncategorized', inplace=True)
dim_prod_df['Sub Category'].fillna('Uncategorized', inplace=True)


For the product database, we first eliminated all rows where the `Product Status` was marked as "Outdated" to focus only on current products. Subsequently, we assigned the value "Uncategorized" to all missing entries in the `Product Category` and `Sub Category` columns to ensure data completeness and enable further analysis.

#### Sales Database:

In [104]:
fact_sales_df.head()

,ProductKey,OrderDateKey,CustomerKey,SalesOrderNumber,SalesAmount
0,381,20220101,16942,SO46700,1000.4375
1,375,20220101,15114,SO46701,2181.5625
2,369,20220101,15116,SO46702,2443.3500
3,337,20220101,20576,SO46703,782.9900
4,370,20220101,13059,SO46704,2443.3500


In [105]:
print(fact_sales_df.isnull().sum()[fact_sales_df.isnull().sum() > 0])

Series([], dtype: int64)


#### Budget Dataset:

In [44]:
print(fact_budget_df.isnull().sum()[fact_budget_df.isnull().sum() > 0])

Series([], dtype: int64)


In [46]:
fact_budget_df.dtypes

Date      object
Budget     int64
dtype: object

In [52]:
fact_budget_df['DateKey'] = pd.to_datetime(fact_budget_df['Date'], format='%B-%y').dt.strftime('%Y%m')
fact_budget_df.drop(columns=['Date'], inplace=True)

In [53]:
fact_budget_df.head()

,Budget,DateKey
0,379665,202201
1,360220,202202
2,391676,202203
3,374843,202204
4,371747,202205


For the budget dataset, we transformed the `Date` column into a standardized `DateKey` format representing the year and month (`YYYYMM`), ensuring compatibility with other datasets. We then dropped the original `Date` column to streamline the dataset, leaving only the relevant `Budget` and `DateKey` columns for further analysis.

In [106]:
fact_budget_df.to_csv('./data/fact_budget.csv', index=False)
fact_sales_df.to_csv('./data/fact_sales.csv', index=False)
dim_prod_df.to_csv('./data/dim_prod.csv', index=False)
dim_cust_df.to_csv('./data/dim_cust.csv', index=False)